In [31]:
import sqlite3
from db_utils import db_connect
import pandas as pd
import os
import csv


In [2]:
con = db_connect() # connect to the database

In [3]:
cur = con.cursor() # instantiate a cursor obj

In [7]:
data_dir = os.path.join('..', 'Resources', 'Data', 'Cleaned_Data')


In [8]:
fips_df = pd.read_csv(os.path.join(data_dir,'fips_codes.csv'))

In [9]:
fips_df

,fips,name,state
0,0,UNITED STATES,NaN
1,1000,ALABAMA,NaN
2,1001,Autauga County,AL
3,1003,Baldwin County,AL
4,1005,Barbour County,AL
...,...,...,...
3190,56037,Sweetwater County,WY
3191,56039,Teton County,WY
3192,56041,Uinta County,WY
3193,56043,Washakie County,WY


In [15]:
fips_df.to_sql('FIPS_Codes', con=con, if_exists='replace', index=False, dtype={'fips': 'INTEGER PRIMARY KEY','Name': 'TEXT NOT NULL', 'state' : 'TEXT'})

In [16]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('FIPS_Codes',)]


In [17]:
pd.read_sql_query ('select * from FIPS_Codes', con)



,fips,name,state
0,0,UNITED STATES,None
1,1000,ALABAMA,None
2,1001,Autauga County,AL
3,1003,Baldwin County,AL
4,1005,Barbour County,AL
...,...,...,...
3190,56037,Sweetwater County,WY
3191,56039,Teton County,WY
3192,56041,Uinta County,WY
3193,56043,Washakie County,WY


In [34]:
columnsQuery = ("PRAGMA table_info([FIPS_Codes])") 
columnNames = cur.execute(columnsQuery)
columnNames.fetchall()

[(0, 'fips', 'INTEGER', 0, None, 1),
 (1, 'name', 'TEXT', 0, None, 0),
 (2, 'state', 'TEXT', 0, None, 0)]

In [23]:
utilities_df = pd.read_csv(os.path.join(data_dir,'master_contaminants.csv'))

In [27]:
utilities_df.county_FIPS = pd.Series(utilities_df.county_FIPS, dtype=pd.Int64Dtype())


In [30]:
utilities_df.dtypes

Utility               object
Contaminant_Factor     int64
Zip                    int64
county_FIPS            Int64
dtype: object

In [29]:
utilities_df.columns

Index(['Utility', 'Contaminant_Factor', 'Zip', 'county_FIPS'], dtype='object')

In [36]:
create_utils_sql = '''
    CREATE TABLE Utilities (
    Utility TEXT NOT NULL, 
    Contaminant_Factor INTEGER NOT NULL, 
    Zip INTEGER NOT NULL, 
    county_FIPS INTEGER NOT NULL,
    FOREIGN KEY (county_FIPS)
       REFERENCES FIPS_Codes (fips) 
);
'''

In [37]:
cur.execute(create_utils_sql)

In [38]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('FIPS_Codes',), ('Utilities',)]


In [44]:
file = open(os.path.join(data_dir,'master_contaminants_noheader.csv'))
rows = csv.reader(file)
cur.executemany("INSERT INTO Utilities VALUES (?, ?, ?, ?)", rows)

In [41]:
move_util_data_sql = '''
    .mode csv
    .import c:/Users/jonat/UO_Bootcamp/Group_project/git_Water_Quality_Analysis/Water_Quality_Analysis/Resources/Data/Cleaned_Data/master_contaminants_noheader.csv
'''

In [42]:
cur.execute(move_util_data_sql)

OperationalError: near ".": syntax error